## Step 1 - Imports
- Import bq, pandas and numpy

In [191]:
import google.datalab.bigquery as bq
import pandas as pd
import numpy as np

## Step 2 - Load Data


### Step 2.1 - Get the matches

In [192]:
%%bq query -n matches
SELECT match,home, away, HomeCode, AwayCode, HomeWin, Draw, AwayWin FROM `project-paul-the-octopus.kanjih.matches` LIMIT 1000

In [193]:
%%bq sample --count 5 --query matches

match,home,away,HomeCode,AwayCode,HomeWin,Draw,AwayWin
1,Egypt,Uruguay,EGY,URU,17,27,56
3,Saudi Arabia,Egypt,KSA,EGY,17,26,57
2,Russia,Egypt,RUS,EGY,49,28,23
1,Russia,Saudi Arabia,RUS,KSA,72,20,8
2,Uruguay,Saudi Arabia,URU,KSA,77,17,6


### Step 2.1 - Get the Fifa Rank

In [194]:
%%bq query -n ranking
SELECT
  rank.team_code,
  rank.Team,
  rank.Total_Points,
  classification.median_goals_scored,
  classification.median_goals_taked,
  classification.around_median_goals_scored,
  classification.around_median_goals_taked
FROM
  `project-paul-the-octopus.kanjih.rank_only_in_wordcup_2018` AS rank,
  `project-paul-the-octopus.kanjih.classification` AS classification
WHERE
  rank.team_code = classification.team_code ORDER BY rank.Total_Points DESC

In [195]:
%%bq sample --count 5 --query ranking

team_code,Team,Total_Points,median_goals_scored,median_goals_taked,around_median_goals_scored,around_median_goals_taked
GER,Germany,1609,4.3,0.4,4,0
BRA,Brazil,1489,2.277777778,0.6111111111,2,1
POR,Portugal,1360,3.2,0.4,3,0
ARG,Argentina,1359,1.055555556,0.6111111111,1,1
BEL,Belgium,1337,4.3,0.6,4,1


### Step 2.2 - Put the data in a dataframe

In [196]:
df_matches = matches.execute(output_options=bq.QueryOutput.dataframe()).result()
len(df_matches) 

48

In [197]:
df_ranking = ranking.execute(output_options=bq.QueryOutput.dataframe()).result()
len(df_ranking)

32

### Step 2.3 - Increase ranking value for world cup  host

In [198]:
df_ranking['Total_Points'] = np.where(df_ranking['team_code'] == 'RUS',  df_ranking['Total_Points'] + 400, df_ranking['Total_Points'])

### Step 2.3 - Merge dataframe to get points from Fifa Rank


In [199]:
df_result = pd.merge(df_matches, df_ranking[['team_code','Total_Points','median_goals_scored','median_goals_taked']], left_on = 'HomeCode', right_on = 'team_code')
df_result.rename(columns={'Total_Points': 'homePoints'}, inplace=True)
df_result.rename(columns={'median_goals_scored': 'homeMedianGoalScored'}, inplace=True)
df_result.rename(columns={'median_goals_taked': 'homeMedianGoalTaked'}, inplace=True)
df_result= pd.merge(df_result, df_ranking[['team_code','Total_Points', 'median_goals_scored','median_goals_taked']], left_on = 'AwayCode', right_on = 'team_code')
df_result.rename(columns={'Total_Points': 'awayPoints'}, inplace=True)
df_result.rename(columns={'median_goals_scored': 'awayMedianGoalScored'}, inplace=True)
df_result.rename(columns={'median_goals_taked': 'awayMedianGoalTaked'}, inplace=True)
df_result['diffPoints'] = (df_result['homePoints'] - df_result['awayPoints']).abs()
df_result = df_result[['match','home','away','HomeCode','AwayCode','HomeWin','Draw', 'AwayWin', 'homePoints', 'awayPoints', 'diffPoints','homeMedianGoalScored', 'homeMedianGoalTaked', 'awayMedianGoalScored', 'awayMedianGoalTaked']]
df = df_result.copy()
df

,match,home,away,HomeCode,AwayCode,HomeWin,Draw,AwayWin,homePoints,awayPoints,diffPoints,homeMedianGoalScored,homeMedianGoalTaked,awayMedianGoalScored,awayMedianGoalTaked
0,1,Tunisia,England,TUN,ENG,10,19,71,920,1047,127,1.833333,0.666667,1.800000,0.300000
1,1,Egypt,Uruguay,EGY,URU,17,27,56,687,931,244,1.333333,0.666667,1.777778,1.111111
2,1,Portugal,Spain,POR,ESP,22,27,51,1360,1228,132,3.200000,0.400000,3.600000,0.300000
3,2,Iran,Spain,IRN,ESP,5,14,81,792,1228,436,1.000000,0.200000,3.600000,0.300000
4,2,Portugal,Morocco,POR,MAR,62,24,14,1360,694,666,3.200000,0.400000,1.833333,0.100000
5,3,Spain,Morocco,ESP,MAR,69,20,11,1228,694,534,3.600000,0.300000,1.833333,0.100000
6,1,Peru,Denmark,PER,DEN,25,28,47,1128,1108,20,1.500000,1.444444,2.000000,0.800000
7,1,Costa Rica,Serbia,CRC,SRB,25,28,47,872,780,92,1.400000,0.800000,2.000000,1.000000
8,1,Morocco,Iran,MAR,IRN,42,31,27,694,792,98,1.833333,0.100000,1.000000,0.200000
9,1,Poland,Senegal,POL,SEN,42,30,28,1228,862,366,2.800000,1.400000,1.666667,0.500000


## Step 3 - Prediction

### Step 3.1 - Predication Layer 1 (Generate winner by score match)

In [200]:
df['homeScore'] = np.where((df["HomeWin"]-df["AwayWin"]) > 10, np.around(1 +(df["homeMedianGoalScored"] * (df["awayMedianGoalTaked"]))), 0)
df['awayScore'] = np.where((df["AwayWin"]-df["HomeWin"]) > 10, np.around(1 + (df["awayMedianGoalScored"] * (df["homeMedianGoalTaked"]) )), 0)
df

,match,home,away,HomeCode,AwayCode,HomeWin,Draw,AwayWin,homePoints,awayPoints,diffPoints,homeMedianGoalScored,homeMedianGoalTaked,awayMedianGoalScored,awayMedianGoalTaked,homeScore,awayScore
0,1,Tunisia,England,TUN,ENG,10,19,71,920,1047,127,1.833333,0.666667,1.800000,0.300000,0.0,2.0
1,1,Egypt,Uruguay,EGY,URU,17,27,56,687,931,244,1.333333,0.666667,1.777778,1.111111,0.0,2.0
2,1,Portugal,Spain,POR,ESP,22,27,51,1360,1228,132,3.200000,0.400000,3.600000,0.300000,0.0,2.0
3,2,Iran,Spain,IRN,ESP,5,14,81,792,1228,436,1.000000,0.200000,3.600000,0.300000,0.0,2.0
4,2,Portugal,Morocco,POR,MAR,62,24,14,1360,694,666,3.200000,0.400000,1.833333,0.100000,1.0,0.0
5,3,Spain,Morocco,ESP,MAR,69,20,11,1228,694,534,3.600000,0.300000,1.833333,0.100000,1.0,0.0
6,1,Peru,Denmark,PER,DEN,25,28,47,1128,1108,20,1.500000,1.444444,2.000000,0.800000,0.0,4.0
7,1,Costa Rica,Serbia,CRC,SRB,25,28,47,872,780,92,1.400000,0.800000,2.000000,1.000000,0.0,3.0
8,1,Morocco,Iran,MAR,IRN,42,31,27,694,792,98,1.833333,0.100000,1.000000,0.200000,1.0,0.0
9,1,Poland,Senegal,POL,SEN,42,30,28,1228,862,366,2.800000,1.400000,1.666667,0.500000,2.0,0.0


### Step 3.2 - Predication Layer 2 (Generate draw by match 1 using fifa ranking difference)

In [201]:
df['homeScore'] = np.where(((df['match'] == 1) & (df['diffPoints'] < 200)), np.around(df['homeMedianGoalScored']) , df['homeScore'] )
df['awayScore'] = np.where(((df['match'] == 1) & (df['diffPoints'] < 200)), np.around(df['homeMedianGoalScored']) , df['awayScore'] )
df

,match,home,away,HomeCode,AwayCode,HomeWin,Draw,AwayWin,homePoints,awayPoints,diffPoints,homeMedianGoalScored,homeMedianGoalTaked,awayMedianGoalScored,awayMedianGoalTaked,homeScore,awayScore
0,1,Tunisia,England,TUN,ENG,10,19,71,920,1047,127,1.833333,0.666667,1.800000,0.300000,2.0,2.0
1,1,Egypt,Uruguay,EGY,URU,17,27,56,687,931,244,1.333333,0.666667,1.777778,1.111111,0.0,2.0
2,1,Portugal,Spain,POR,ESP,22,27,51,1360,1228,132,3.200000,0.400000,3.600000,0.300000,3.0,3.0
3,2,Iran,Spain,IRN,ESP,5,14,81,792,1228,436,1.000000,0.200000,3.600000,0.300000,0.0,2.0
4,2,Portugal,Morocco,POR,MAR,62,24,14,1360,694,666,3.200000,0.400000,1.833333,0.100000,1.0,0.0
5,3,Spain,Morocco,ESP,MAR,69,20,11,1228,694,534,3.600000,0.300000,1.833333,0.100000,1.0,0.0
6,1,Peru,Denmark,PER,DEN,25,28,47,1128,1108,20,1.500000,1.444444,2.000000,0.800000,2.0,2.0
7,1,Costa Rica,Serbia,CRC,SRB,25,28,47,872,780,92,1.400000,0.800000,2.000000,1.000000,1.0,1.0
8,1,Morocco,Iran,MAR,IRN,42,31,27,694,792,98,1.833333,0.100000,1.000000,0.200000,2.0,2.0
9,1,Poland,Senegal,POL,SEN,42,30,28,1228,862,366,2.800000,1.400000,1.666667,0.500000,2.0,0.0


## STEP 4 - Clean Data

In [204]:
df_to_print = df[['home','homeScore', 'awayScore','away']].copy()
df_to_print.rename(columns={'homeScore': 'home_score'}, inplace=True)
df_to_print.rename(columns={'awayScore': 'away_score'}, inplace=True)
pd.options.display.float_format = '{:,.0f}'.format
df_to_print

,home,home_score,away_score,away
0,Tunisia,2,2,England
1,Egypt,0,2,Uruguay
2,Portugal,3,3,Spain
3,Iran,0,2,Spain
4,Portugal,1,0,Morocco
5,Spain,1,0,Morocco
6,Peru,2,2,Denmark
7,Costa Rica,1,1,Serbia
8,Morocco,2,2,Iran
9,Poland,2,0,Senegal


## Step 5 - Print CSV

In [205]:
%%gcs write --variable df_to_print --object 'gs://paul_kanjih_v1/predictions.csv'